# Guía 1: Introducción a los Modelos de Lenguaje

## Descripción

En esta guía vas a tener tu primer contacto con los Modelos de Lenguaje de Gran Escala (LLMs, por sus siglas en inglés). Vamos a trabajar con **Phi-3**, un modelo desarrollado por Microsoft que entiende y genera texto en español de manera natural.

Al finalizar esta guía vas a poder:
- Cargar y ejecutar un modelo de lenguaje preentrenado
- Generar texto coherente en español usando prompts
- Comprender los parámetros básicos de generación de texto
- Experimentar con diferentes tipos de instrucciones

---

## Configuración del Entorno

### Requisitos

Para ejecutar este notebook necesitás:

1. **Google Colab con GPU activada:**
   - Andá a `Entorno de ejecución > Cambiar tipo de entorno de ejecución`
   - Seleccioná `Acelerador de Hardware > GPU`
   - Elegí `Tipo de GPU > T4`

2. **Conexión a Internet** para descargar el modelo y las bibliotecas

La GPU T4 es suficiente para ejecutar Phi-3-mini y es gratuita en Colab.

## Instalación de Dependencias

Primero instalamos las bibliotecas necesarias:

- **transformers**: Biblioteca principal de Hugging Face para trabajar con modelos de lenguaje
- **accelerate**: Biblioteca para acelerar el entrenamiento e inferencia de modelos

Usamos versiones específicas para asegurar compatibilidad.

In [1]:
# Instalación de dependencias
# El prefijo ! permite ejecutar comandos de sistema desde el notebook
!pip install transformers>=4.40.1 accelerate>=0.27.2

## Marco Teórico

### ¿Qué es un Modelo de Lenguaje?

Un **modelo de lenguaje** es un sistema que aprendió a predecir y generar texto a partir de haber procesado millones de ejemplos durante su entrenamiento. Estos modelos capturan patrones del lenguaje natural: gramática, sintaxis, semántica y hasta conocimiento factual sobre el mundo.

### ¿Qué es Phi-3?

**Phi-3-mini-4k-instruct** es un modelo de lenguaje desarrollado por Microsoft con las siguientes características:

- **Tamaño**: 3.8 mil millones de parámetros (considerado "pequeño" para estándares actuales)
- **Contexto**: Puede procesar hasta 4096 tokens (aproximadamente 3000 palabras)
- **Multilingüe**: Entrenado en múltiples idiomas, incluyendo español
- **Instruction-tuned**: Ajustado específicamente para seguir instrucciones del usuario
- **Capacidades**: Escritura creativa, respuestas a preguntas, análisis de texto, traducción, y más

### ¿Cómo funciona la generación de texto?

El modelo funciona en tres pasos:

1. **Entrada (Input)**: Recibe un texto inicial llamado "prompt" o "instrucción"
2. **Procesamiento**: Analiza el contexto usando su conocimiento aprendido durante el entrenamiento
3. **Generación (Output)**: Produce una continuación coherente del texto, palabra por palabra

El modelo no "entiende" como los humanos, pero aprendió patrones estadísticos tan complejos que genera texto sorprendentemente coherente y contextualmente apropiado.

## Carga del Modelo

Ahora vamos a cargar el modelo Phi-3 en la memoria de la GPU.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Cargamos el modelo Phi-3-mini
# Este proceso puede tardar 1-2 minutos la primera vez
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",  # Identificador del modelo en Hugging Face
    device_map="cuda",                    # device_map: Dónde cargar el modelo ("cuda" = GPU, "cpu" = procesador)
    torch_dtype="auto",                   # torch_dtype: Tipo de datos numéricos ("auto" selecciona el óptimo)
    trust_remote_code=False,              # trust_remote_code: Por seguridad, no ejecutar código remoto no verificado
)

# Cargamos el tokenizador correspondiente al modelo
# El tokenizador convierte texto en números que el modelo puede procesar
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

print("Modelo cargado exitosamente")

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Modelo cargado exitosamente


### Explicación de Componentes

**AutoModelForCausalLM**: Clase que carga modelos de lenguaje causal (generan texto de izquierda a derecha).

**AutoTokenizer**: Convierte texto en tokens (unidades mínimas que el modelo procesa). Cada modelo tiene su propio tokenizador específico.

**Parámetros de carga:**

- `device_map`: Controla en qué dispositivo se carga el modelo
  - `"cuda"`: Usa la GPU (más rápido)
  - `"cpu"`: Usa el procesador (más lento pero no requiere GPU)
  - `"auto"`: Decide automáticamente

- `torch_dtype`: Precisión numérica de los cálculos
  - `"auto"`: Selecciona automáticamente (recomendado)
  - `torch.float16`: Menor precisión, más rápido, usa menos memoria
  - `torch.float32`: Mayor precisión, más lento, usa más memoria

- `trust_remote_code`: Por seguridad, mantener en `False` a menos que confíes explícitamente en el modelo

## Creación del Pipeline de Generación

Un **pipeline** simplifica el uso del modelo encapsulando todos los pasos necesarios (tokenización, inferencia, decodificación) en una sola función.

In [3]:
from transformers import pipeline

# Creamos un pipeline de generación de texto
generador = pipeline(
    "text-generation",           # Tipo de tarea
    model=model,                  # Modelo a usar
    tokenizer=tokenizer,          # Tokenizador a usar
    return_full_text=False,       # return_full_text: Si False, solo devuelve el texto generado (no el prompt)
    max_new_tokens=500,           # max_new_tokens: Cantidad máxima de tokens a generar
    do_sample=False               # do_sample: Si False, usa generación determinística (siempre la misma salida)
)

print("Pipeline creado exitosamente")

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Pipeline creado exitosamente


### Parámetros del Pipeline

- `return_full_text`:
  - `False`: Solo devuelve el texto nuevo generado
  - `True`: Devuelve el prompt original + el texto generado

- `max_new_tokens`: Límite de tokens (palabras/subpalabras) a generar
  - Valores típicos: 50-500 para respuestas cortas, 500-2000 para textos largos
  - Afecta el tiempo de generación: más tokens = más tiempo

- `do_sample`: Controla el tipo de generación
  - `False`: Determinística (greedy decoding) - siempre elige la palabra más probable
  - `True`: Estocástica (sampling) - introduce aleatoriedad para mayor creatividad

## Primer Ejemplo: Generación de Texto

Ahora sí, vamos a generar nuestro primer texto.

In [4]:
# Definimos el mensaje como una conversación
# Phi-3 está entrenado para recibir mensajes en formato de conversación
mensaje = [
    {"role": "user", "content": "Escribí un párrafo corto sobre el dulce de leche y su importancia en la cultura argentina."}
]

# Generamos la respuesta
salida = generador(mensaje)

# Mostramos el resultado
print(salida[0]["generated_text"])

 El dulce de leche, conocido también como "manjar" en Argentina, es una delicia que ha trascendido fronteras y generaciones. Este dulce, con su sabor rico y sedoso, se elabora a partir de la cocción prolongada de leche condensada, lo que le confiere su característico color amarillo dorado y su textura cremosa. No solo es un postre popular en reuniones familiares y fiestas, sino que también se utiliza en una variedad de platos argentinos, desde los tradicionales empanadas hasta los famosos locos de leche. La importancia del dulce de leche en la cultura argentina es incuestionable, ya que representa un símbolo de la identidad culinaria del país y es un elemento esencial en la gastronomía argentina.


### Análisis del Formato de Mensaje

Los modelos instruction-tuned como Phi-3 esperan mensajes estructurados:

```python
mensaje = [
    {"role": "user", "content": "tu instrucción aquí"}
]
```

- `role`: Quién habla ("user" = usuario, "assistant" = modelo)
- `content`: El texto del mensaje

Este formato permite simular conversaciones multi-turno agregando más mensajes a la lista.

## Ejercicios Prácticos

Ahora te toca experimentar. Probá modificar el contenido del mensaje para diferentes tareas:

In [5]:
# Ejercicio 1: Instrucciones procedimentales
mensaje_1 = [
    {"role": "user", "content": "Escribí una receta de mate paso a paso"}
]

salida_1 = generador(mensaje_1)
print("=== EJERCICIO 1: RECETA DE MATE ===")
print(salida_1[0]["generated_text"])
print("\n")

=== EJERCICIO 1: RECETA DE MATE ===
 **Receta de Mate:**


Ingredientes:

- 1 mate (recipiente para el té)

- 10 gramos de yerba mate

- Agua tibia

- Taza pequeña

- Cucharón


Pasos:

1. Llena el mate con la yerba mate hasta la mitad.

2. Vierte agua tibia en el mate hasta que la yerba esté completamente sumergida.

3. Deja reposar el mate durante 3 a 5 minutos para que la infusión se desarrolle.

4. Usa el cucharón para remover la yerba y el mate para distribuir la infusión de manera uniforme.

5. Sírvelo en una taza pequeña y disfruta de tu mate.


Recuerda que la temperatura del agua es crucial para el sabor del mate. No debe ser demasiado caliente ni demasiado fría.




In [6]:
# Ejercicio 2: Análisis comparativo
mensaje_2 = [
    {"role": "user", "content": "Explicá la diferencia entre el asado argentino y otros tipos de BBQ"}
]

salida_2 = generador(mensaje_2)
print("=== EJERCICIO 2: ASADO VS BBQ ===")
print(salida_2[0]["generated_text"])
print("\n")

=== EJERCICIO 2: ASADO VS BBQ ===
 El asado argentino es una tradición culinaria que se ha convertido en un símbolo de la cultura argentina. A diferencia de otros tipos de barbacoa, el asado argentino se caracteriza por su técnica de cocción al aire libre y su variedad de cortes de carne.


En el asado argentino, se utilizan parrillas de carbón, conocidas como "parrillas", que se colocan al aire libre, generalmente en un lugar abierto y con viento. La carne se coloca en la parrilla y se cocina lentamente, permitiendo que el calor del carbón se disperse de manera uniforme. Este método de cocción le da a la carne un sabor distintivo y una textura tierna.


Otros tipos de barbacoa pueden variar en técnica y ingredientes. Por ejemplo, en la barbacoa mexicana, la carne se cocina en una olla de barro con especias y hierbas, mientras que en la barbacoa de los Estados Unidos, la carne suele ser asada en una parrilla de gas o carbón con una mezcla de especias y condimentos.


Además, el asado a

In [7]:
# Ejercicio 3: Generación creativa con contexto local
mensaje_3 = [
    {"role": "user", "content": "Escribí un diálogo breve usando modismos porteños"}
]

salida_3 = generador(mensaje_3)
print("=== EJERCICIO 3: DIÁLOGO PORTEÑO ===")
print(salida_3[0]["generated_text"])

=== EJERCICIO 3: DIÁLOGO PORTEÑO ===
 **Personaje 1:** ¡Ahí viene el tango, mi amor!

**Personaje 2:** ¡Claro que sí! Pero no se me olvide, tengo que ir al 'cabaré' a ver si puedo encontrarme con mi 'pibe'.

**Personaje 1:** ¡No te preocupes, que te espera allí! Y mientras tanto, ¿te gustaría tomar un'mate' en la 'parrilla'?

**Personaje 2:** ¡Perfecto! Me encanta el'mate' con un poco de 'chimichurri'.

**Personaje 1:** Genial, entonces, ¡vamos a 'darle la vuelta al mundo' con el tango y luego al'mate'!

**Personaje 2:** ¡Eso es lo que busco! ¡Vamos a 'llegar a la cima' de la noche porteña!


## Experimentación Libre

Usá la celda siguiente para probar tus propios prompts:

In [ ]:
# Escribí tu propio prompt aquí
mi_mensaje = [
    {"role": "user", "content": "Tu instrucción aquí"}
]

mi_salida = generador(mi_mensaje)
print(mi_salida[0]["generated_text"])

## Consejos para Mejores Resultados

### Técnicas de Prompt Engineering

1. **Sé específico**: Instrucciones claras generan mejores resultados
   - Malo: "Hablame del mate"
   - Bueno: "Explicá en 3 párrafos la historia del mate en Argentina"

2. **Incluí contexto**: El modelo funciona mejor con información contextual
   - "Escribí un email formal para un profesor explicando una ausencia"

3. **Especificá el formato**: Indicá cómo querés la respuesta
   - "Listá 5 ventajas del transporte público en formato de viñetas"

4. **Usá ejemplos**: Si buscás un estilo particular, dá un ejemplo
   - "Escribí una crítica cinematográfica al estilo de los medios especializados"

5. **Experimentá con temperatura**: Cuando uses `do_sample=True`, podés controlar la creatividad
   - Baja temperatura (0.1-0.5): Respuestas más conservadoras y predecibles
   - Alta temperatura (0.7-1.0): Respuestas más creativas y variadas

## Resumen de Conceptos Clave

En esta guía aprendiste:

1. **Modelos de Lenguaje**: Sistemas entrenados para generar y comprender texto natural

2. **Phi-3**: Modelo compacto pero poderoso de Microsoft (3.8B parámetros)

3. **Componentes esenciales**:
   - `AutoModelForCausalLM`: Carga el modelo
   - `AutoTokenizer`: Convierte texto en tokens
   - `pipeline`: Simplifica la generación de texto

4. **Parámetros importantes**:
   - `device_map`: Dónde ejecutar el modelo (CPU/GPU)
   - `max_new_tokens`: Longitud máxima de la respuesta
   - `do_sample`: Generación determinística vs. estocástica
   - `return_full_text`: Incluir o no el prompt en la salida

5. **Formato de mensaje**: Estructura JSON con `role` y `content`

6. **Prompt Engineering**: Técnicas para obtener mejores resultados del modelo

## Guía de Preguntas y Respuestas

### Preguntas de Comprensión

**1. ¿Qué es un modelo de lenguaje y cómo aprende a generar texto?**

<details>
<summary>Ver respuesta</summary>

Un modelo de lenguaje es un sistema de inteligencia artificial que fue entrenado con millones de ejemplos de texto para aprender patrones del lenguaje natural. Durante el entrenamiento, el modelo ajustó sus parámetros internos (como neuronas en una red) para predecir qué palabra viene después en una secuencia de texto. Esto le permite generar texto coherente porque aprendió gramática, sintaxis, semántica y conocimiento factual de los datos de entrenamiento.
</details>

---

**2. ¿Cuál es la diferencia entre usar `device_map="cuda"` y `device_map="cpu"`?**

<details>
<summary>Ver respuesta</summary>

- `device_map="cuda"`: Carga el modelo en la GPU (tarjeta gráfica), lo que resulta en generación de texto mucho más rápida (típicamente 10-50x más rápido). Requiere tener una GPU compatible con CUDA.

- `device_map="cpu"`: Carga el modelo en el procesador principal (CPU), lo que es más lento pero funciona en cualquier computadora sin necesidad de GPU especial. Útil cuando no tenés acceso a GPU o para modelos muy pequeños.
</details>

---

**3. ¿Qué controla el parámetro `max_new_tokens` y cómo afecta la generación?**

<details>
<summary>Ver respuesta</summary>

`max_new_tokens` establece la cantidad máxima de tokens (palabras o subpalabras) que el modelo puede generar en su respuesta. Por ejemplo:

- `max_new_tokens=50`: Respuestas breves (1-2 párrafos)
- `max_new_tokens=500`: Respuestas extensas (varios párrafos)

Afecta:
- **Longitud de la salida**: Limita cuánto puede escribir el modelo
- **Tiempo de generación**: Más tokens = más tiempo de procesamiento
- **Completitud**: Si el límite es muy bajo, la respuesta puede cortarse a mitad de frase
</details>

---

**4. ¿Qué diferencia hay entre `do_sample=True` y `do_sample=False`?**

<details>
<summary>Ver respuesta</summary>

- `do_sample=False` (generación determinística/greedy): El modelo siempre elige la palabra con mayor probabilidad. Genera la misma respuesta cada vez que se ejecuta con el mismo prompt. Recomendado para tareas que requieren consistencia.

- `do_sample=True` (generación estocástica/sampling): El modelo introduce aleatoriedad controlada, eligiendo entre varias palabras posibles según sus probabilidades. Genera respuestas diferentes en cada ejecución. Útil para tareas creativas como escritura de historias o generación de múltiples alternativas.
</details>

---

**5. ¿Por qué usamos un formato de mensaje con `role` y `content` en lugar de solo texto?**

<details>
<summary>Ver respuesta</summary>

Los modelos instruction-tuned como Phi-3 fueron entrenados específicamente con conversaciones estructuradas. El formato:

```python
[{"role": "user", "content": "texto"}]
```

permite:
- **Distinguir roles**: El modelo sabe quién habla (usuario vs. asistente)
- **Conversaciones multi-turno**: Podés agregar múltiples mensajes para simular un diálogo
- **Mejor rendimiento**: El modelo fue específicamente entrenado con este formato, por lo que entiende mejor las instrucciones

Ejemplo de conversación multi-turno:
```python
[
    {"role": "user", "content": "¿Qué es un modelo de lenguaje?"},
    {"role": "assistant", "content": "Un modelo de lenguaje es..."},
    {"role": "user", "content": "¿Podés dar un ejemplo?"}
]
```
</details>

---

### Preguntas de Aplicación

**6. ¿Cómo modificarías el código para generar respuestas más largas y detalladas?**

<details>
<summary>Ver respuesta</summary>

Para respuestas más largas, incrementá el valor de `max_new_tokens` en el pipeline:

```python
generador = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=1000,  # Aumentado de 500 a 1000
    do_sample=False
)
```

También podés ser más específico en tu prompt:
```python
mensaje = [
    {"role": "user", "content": "Escribí un ensayo detallado de al menos 500 palabras sobre..."}
]
```
</details>

---

**7. Si el modelo genera texto que se corta abruptamente, ¿cuál es probablemente el problema y cómo lo solucionarías?**

<details>
<summary>Ver respuesta</summary>

El problema más probable es que `max_new_tokens` es muy bajo y el modelo alcanzó el límite antes de terminar su respuesta.

Soluciones:
1. Aumentá `max_new_tokens` a un valor mayor (ej: 500, 1000)
2. Hacé el prompt más específico pidiendo respuestas más concisas si no querés aumentar el límite
3. Dividí la tarea en múltiples prompts más pequeños
</details>

---

**8. ¿Qué prompt usarías para que el modelo genere una lista de elementos en lugar de un párrafo?**

<details>
<summary>Ver respuesta</summary>

Sé explícito sobre el formato deseado:

```python
mensaje = [
    {"role": "user", "content": "Listá 5 características del mate argentino. Presentá tu respuesta como una lista numerada."}
]
```

O con viñetas:
```python
mensaje = [
    {"role": "user", "content": "Enumerá las ventajas del transporte público. Usá viñetas (-)."}
]
```
</details>

---

**9. Si querés que el modelo sea más creativo y genere respuestas variadas cada vez, ¿qué parámetro modificarías y cómo?**

<details>
<summary>Ver respuesta</summary>

Cambiá `do_sample` a `True` y podés agregar `temperature`:

```python
generador = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=True,         # Activar sampling
    temperature=0.8         # Controlar creatividad (0.1-2.0)
)
```

- `temperature` baja (0.1-0.5): Más conservador
- `temperature` media (0.6-0.9): Balance entre creatividad y coherencia
- `temperature` alta (1.0-2.0): Muy creativo pero puede ser menos coherente
</details>

---

**10. ¿Cómo harías para que el modelo continúe una conversación en lugar de empezar de cero?**

<details>
<summary>Ver respuesta</summary>

Agregá todos los mensajes anteriores a la lista:

```python
conversacion = [
    {"role": "user", "content": "¿Cuál es la capital de Argentina?"},
    {"role": "assistant", "content": "La capital de Argentina es Buenos Aires."},
    {"role": "user", "content": "¿Cuántos habitantes tiene?"}
]

salida = generador(conversacion)
```

El modelo usa todo el contexto previo para generar una respuesta coherente con la conversación.
</details>

---

### Preguntas de Reflexión

**11. ¿Por qué creés que es importante usar ejemplos con contexto cultural argentino al trabajar con estos modelos en un curso local?**

<details>
<summary>Ver respuesta</summary>

Usar contexto cultural local es importante por varias razones:

1. **Relevancia**: Los ejemplos son más significativos y comprensibles para los estudiantes
2. **Evaluación de calidad**: Es más fácil juzgar si el modelo respondió bien cuando conocés el tema
3. **Identificar sesgos**: Podés detectar si el modelo tiene conocimientos limitados o incorrectos sobre la cultura local
4. **Aplicaciones prácticas**: Los estudiantes ven cómo aplicar la tecnología a problemas y contextos reales de su entorno
5. **Testing del modelo**: Evalúa qué tan bien el modelo maneja variantes del español (rioplatense) y conocimiento regional
</details>

---

**12. ¿Qué limitaciones observás en los modelos de lenguaje basándote en lo que experimentaste en esta guía?**

<details>
<summary>Ver respuesta</summary>

Limitaciones comunes:

1. **No razonan realmente**: Generan texto basándose en patrones estadísticos, no en comprensión profunda
2. **Pueden inventar información**: Generan datos falsos con confianza ("alucinaciones")
3. **Conocimiento limitado**: Desactualizados después de su fecha de entrenamiento
4. **Sesgos**: Reflejan sesgos presentes en sus datos de entrenamiento
5. **Inconsistencia**: Pueden dar respuestas diferentes a la misma pregunta
6. **Dependencia del prompt**: La calidad de la salida depende fuertemente de cómo formulás la pregunta
7. **Contexto cultural limitado**: Pueden tener menos conocimiento sobre culturas y lenguas menos representadas en sus datos de entrenamiento
</details>

---

### Desafíos Adicionales

**Desafío 1**: Escribí tres prompts diferentes que generen: (1) un texto formal académico, (2) un texto informal conversacional, y (3) un texto técnico instructivo.

**Desafío 2**: Experimentá con `max_new_tokens` usando valores de 50, 200 y 500 con el mismo prompt. ¿Cómo cambia la calidad y completitud de las respuestas?

**Desafío 3**: Diseñá una conversación multi-turno de al menos 4 intercambios sobre un tema técnico de tu interés.

---

## Referencias y Recursos Adicionales

- [Documentación oficial de Phi-3](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)
- [Hugging Face Transformers](https://huggingface.co/docs/transformers/index)
- [Guía de Prompt Engineering](https://www.promptingguide.ai/es)